In [ ]:
import pandas as pd
import json
import ast
import numpy as np

# Load the ROI data from the JSON file
with open('front_room_roi.json', 'r') as f:
    roi_data = json.load(f)

# Extract all ROI polygons and calculate their bounding boxes
roi_rectangles = {}
for roi_name, points_dict in roi_data['roi'].items():
    points = np.array(list(points_dict.values()))
    min_x, min_y = np.min(points, axis=0)
    max_x, max_y = np.max(points, axis=0)
    roi_rectangles[roi_name] = {'min_x': min_x, 'min_y': min_y, 'max_x': max_x, 'max_y': max_y}

# Load the detections log
df_detections = pd.read_csv('detections_log.csv')

# Convert the string representation of person_xy to a list of floats
df_detections['person_xy'] = df_detections['person_xy'].apply(ast.literal_eval)

# Function to check if a point is inside a given bounding box
def is_point_in_roi(point_xy, rect_min_x, rect_min_y, rect_max_x, rect_max_y):
    x, y = point_xy
    return rect_min_x <= x <= rect_max_x and rect_min_y <= y <= rect_max_y

# Prepare a dictionary to store total time for each ROI
total_time_per_roi = {roi_name: 0.0 for roi_name in roi_rectangles.keys()}

# Process the data frame, grouping by video_time_sec to handle frame by frame
grouped_by_frame = df_detections.groupby('video_time_sec')

previous_video_time_sec = None

for video_time_sec, frame_data in grouped_by_frame:
    
    # Calculate the duration of the current frame
    frame_duration = 0
    if previous_video_time_sec is not None:
        frame_duration = video_time_sec - previous_video_time_sec
    previous_video_time_sec = video_time_sec

    # Find which ROIs have a person in the current frame
    rois_with_person = set()
    for _, person_data in frame_data.iterrows():
        person_xy = person_data['person_xy']
        # Check if the current person is in any ROI
        for roi_name, rect_coords in roi_rectangles.items():
            if is_point_in_roi(person_xy, rect_coords['min_x'], rect_coords['min_y'], rect_coords['max_x'], rect_coords['max_y']):
                rois_with_person.add(roi_name)
    
    # Add the frame duration to the total time for each ROI that had a person
    for roi_name in rois_with_person:
        total_time_per_roi[roi_name] += frame_duration

# Create the final roi_log DataFrame
roi_log = pd.DataFrame(total_time_per_roi.items(), columns=['ROI Booth', 'Total Seconds'])

print("Final roi_log table:")
print(roi_log)